In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("East_Massachusetts_Jan_PM");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("East_Massachusetts_trips_Jan_PM.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 64 entries:
  (3,6) => 5000.0
  (4,4) => 0.0
  (3,1) => 6000.0
  (4,5) => 700.0
  (2,4) => 4000.0
  (6,5) => 6000.0
  (8,4) => 3000.0
  (1,2) => 1000.0
  (5,1) => 1500.0
  (3,4) => 2000.0
  (8,2) => 2000.0
  (2,3) => 1000.0
  (2,6) => 900.0
  (5,4) => 7000.0
  (2,1) => 1500.0
  (2,5) => 1400.0
  (8,5) => 1000.0
  (1,4) => 2000.0
  (2,8) => 6000.0
  (5,8) => 3000.0
  (4,8) => 2500.0
  (7,8) => 2000.0
  (1,1) => 0.0
  (7,7) => 0.0
  (5,7) => 2000.0
  ⋮     => ⋮

In [7]:
odPairLabel_

Dict{String,Any} with 56 entries:
  "32" => {5,4}
  "29" => {5,1}
  "1"  => {1,2}
  "54" => {8,5}
  "2"  => {1,3}
  "41" => {6,7}
  "53" => {8,4}
  "51" => {8,2}
  "27" => {4,7}
  "42" => {6,8}
  "33" => {5,6}
  "28" => {4,8}
  "52" => {8,3}
  "50" => {8,1}
  "26" => {4,6}
  "10" => {2,4}
  "24" => {4,3}
  "25" => {4,5}
  "23" => {4,2}
  "49" => {7,8}
  "5"  => {1,6}
  "31" => {5,3}
  "43" => {7,1}
  "39" => {6,4}
  "44" => {7,2}
  ⋮    => ⋮

In [8]:
demandsVec = zeros(56)

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

56-element Array{Float64,1}:
 1000.0
 4000.0
 2000.0
 4000.0
 1500.0
 3000.0
 2500.0
 1500.0
 1000.0
 4000.0
 1400.0
  900.0
 1000.0
    ⋮  
 8000.0
 3000.0
 1500.0
 1500.0
 2000.0
 4000.0
 2000.0
 9000.0
 3000.0
 1000.0
 1500.0
 3000.0

In [10]:
for key=keys(odPairRoute)
    if contains(key, "56-")
        println(key)
    end
end

56-724
56-721
56-719
56-717
56-718
56-715
56-722
56-723
56-713
56-714
56-720
56-716


In [11]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 24
numRoute = 724
numOD = 56

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4092
Number of nonzeros in inequality constraint Jacobian.:      724
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:      748
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [12]:
getValue(linkFlow)

24-element Array{Float64,1}:
  8879.4 
  7220.93
  9120.6 
 16279.1 
 10607.6 
 11220.9 
 11571.8 
 10000.0 
 19893.7 
 25415.4 
 18334.5 
 20584.6 
 12700.0 
 17000.0 
  6000.0 
 11000.0 
  8071.75
  9000.0 
 21693.7 
 14915.4 
 17434.5 
 13584.6 
 16428.2 
 14500.0 

In [13]:
getObjectiveValue(m)

90365.12823966307

In [14]:
outfile = open("flows_converge.txt", "w")

write(outfile, join(("From", "to", "Volume_Capacity"), "        "), "\n")

for i = 1:length(ta_data.start_node)
     n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], getValue(linkFlow)[i]
     write(outfile, join((n1, n2, n3), "        "), "\n")
end

close(outfile)

In [15]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 24 entries:
  (3,6) => 18334.499732188626
  (2,3) => 10607.647137162887
  (5,6) => 5999.99999922194
  (5,4) => 16999.99999925986
  (2,1) => 7220.92708574982
  (6,3) => 20584.59530831657
  (3,1) => 16279.072912800266
  (1,3) => 9120.602171834937
  (4,5) => 12699.9999992296
  (4,8) => 8071.750688885729
  (7,5) => 14915.404690649051
  (8,7) => 14499.99999917247
  (3,2) => 11220.927086100042
  (7,6) => 13584.59530809483
  (2,4) => 11571.750688806444
  (7,8) => 16428.249309239713
  (6,5) => 10999.999999179643
  (6,7) => 17434.499732009182
  (8,4) => 8999.99999895298
  (1,2) => 8879.397826715158
  (5,3) => 25415.404690671494
  (4,2) => 9999.999998903959
  (3,5) => 19893.749576896924
  (5,7) => 21693.749576801943

In [16]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 24  # number of links
numNodes = 8  # number of nodes

@defVar(m, linkFlow[1:numLinks])

@defVar(m, odLinkFlow[keys(demands), 1:numLinks])
for (s,t) = keys(demands)
    for k = 1:numLinks
        @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
    end
end

for k = 1:numLinks
    linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
    @addConstraint(m, linkFlowPartial == linkFlow[k])
end

for l = 1:numNodes
    for (s,t) = keys(demands)
        if s != t
            odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
            if (l == s)
                @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
            elseif (l == t)
                @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
            else
                @addConstraint(m, odLinkFlowPartial == 0)
            end
        end
    end
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:24})

@setNLObjective(m, Min, f)

solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4248
Number of nonzeros in inequality constraint Jacobian.:     1536
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:     1560
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      472
Total number of inequality constraints...............:     1536
        inequality constraints with only lower bounds:     1536
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [17]:
getValue(linkFlow)

24-element Array{Float64,1}:
 10693.0 
  3433.6 
 12807.0 
 14566.4 
 14740.0 
  4300.0 
  9953.09
 12433.6 
 24662.8 
 20064.0 
 21384.1 
 17302.4 
 17000.0 
 12700.0 
 11000.0 
  6000.0 
  8953.09
  8933.6 
 14162.8 
 21864.0 
 14384.1 
 16402.4 
 14546.9 
 15566.4 

In [18]:
getObjectiveValue(m)

89966.31872423957

In [19]:
8879.4 
  7220.93
  9120.6 
 16279.1 
 10607.6 
 11220.9 
 11571.8 
 10000.0 
 19893.7 
 25415.4 
 18334.5 
 20584.6 
 12700.0 
 17000.0 
  6000.0 
 11000.0 
  8071.75
  9000.0 
 21693.7 
 14915.4 
 17434.5 
 13584.6 
 16428.2 
 14500.0 

LoadError: syntax: extra token "Array" after end of expression
while loading In[19], in expression starting on line 1